##Inference log Convert to gejson

Bounding Box Coordinate
<br/>Class Bounding Box Coordinate

#Bounding Box Coordinate

In [ ]:
pwd

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate


In [ ]:
cd ..

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
!pip install geojson

In [ ]:
import os
import shutil
import pprint
from glob import glob
from termcolor import colored

In [ ]:
import re
import math
import geojson
import rasterio
from osgeo import gdal
import numpy as np

In [ ]:
def make_bbox_paths(folder_name, file_extension):
  
  bbox_paths = glob(folder_name + file_extension)
  bbox_paths = sorted(bbox_paths, reverse=False)
  bbox_paths = sorted(bbox_paths, key=len)

  return bbox_paths

In [ ]:
def make_image_paths(folder_name, bbox_paths):

  image_paths = []
  for bbox_path in bbox_paths:
    img_path = bbox_path.replace('tif.log', 'tif')
    image_path = folder_name + img_path.split('/')[-1]
    image_paths.append(image_path)

  return image_paths

In [ ]:
def make_geojson_paths(folder_name, image_paths):

  geojson_paths = []
  for image_path in image_paths:
    geo_path = image_path.replace('tif', 'geojson')
    geojson_path = folder_name + geo_path.split('/')[-1]
    geojson_paths.append(geojson_path)

  return geojson_paths

In [ ]:
def init_geo_dictionary():

  geo_dictionary = {
  "type": "FeatureCollection",
  "name": "fair1m",
  "crs": { "type": "name", "properties": { "name": "" } },
  "features": [
  { "type": "Feature", "properties": { "id": 1 }, "geometry": { "type": "MultiPolygon",
                                                              "coordinates": [ [ ] ] } }
  ]
  }
  
  return geo_dictionary

In [ ]:
def init_bbox_coordinate(bbox_path):

  result = []

  with open(bbox_path , 'r') as f:
    while True:
      line = f.readline()
      if not line: break
      a = re.findall(r"[-+]?\d*\.\d+|\d+", line)   
      result.append([float(a[i]) for i in range(len(a)) if (i%3 == 2) or (i%3 == 0) and (i != 0)])
    f.close()

  return result

In [ ]:
def digit_length(n):
    return int(math.log10(n)) + 1 if n else 0

In [ ]:
def make_geo_dictionary(image_path, result, geo_dictionary):
  
  ds = gdal.Open(image_path)
  metadata = gdal.Info(ds, format='json')

  pixel_size = metadata['size']
  cornerCoordinates = metadata['cornerCoordinates']
  pixel_h = metadata['size'][0]
  pixel_w = metadata['size'][1]

  lowerLeft = cornerCoordinates['lowerLeft']
  lowerRight = cornerCoordinates['lowerRight']
  upperLeft = cornerCoordinates['upperLeft']
  upperRight = cornerCoordinates['upperRight']

  latitude = lowerLeft[0]
  latitude2 = lowerRight[0]
  longtitude = upperLeft[1]
  longtitude2 = lowerLeft[1]
    
  wgs_h = abs(latitude - latitude2)
  wgs_w = abs(longtitude - longtitude2)

  epsg = re.findall(r'\d+', metadata['coordinateSystem']['wkt'])

  if digit_length(abs(latitude)) >= 5:
    geo_dictionary['crs']['properties']['name'] = 'EPSG:' + epsg[-1]
  
  elif digit_length(abs(latitude)) < 5:
     geo_dictionary['crs']['properties']['name'] = "urn:ogc:def:crs:OGC:1.3:CRS84"

  for each_bbox in result:
    x1 = each_bbox[0]; y1 = each_bbox[1]
    x2 = each_bbox[2]; y2 = each_bbox[3]
    x3 = each_bbox[4]; y3 = each_bbox[5]
    x4 = each_bbox[6]; y4 = each_bbox[7]

    transform_x1 = latitude + ((x1 / pixel_w) * wgs_w)
    transform_y1 = longtitude - ((y1 / pixel_h) * wgs_h)
    transform_x2 = latitude + ((x2 / pixel_w) * wgs_w)
    transform_y2 = longtitude - ((y2 / pixel_h) * wgs_h)    
    transform_x3 = latitude + ((x3 / pixel_w) * wgs_w)
    transform_y3 = longtitude - ((y3 / pixel_h) * wgs_h)
    transform_x4 = latitude + ((x4 / pixel_w) * wgs_w)
    transform_y4 = longtitude - ((y4 / pixel_h) * wgs_h)

    point_count = 0
    for key, value in geo_dictionary.items():
      point_count += 1
      if point_count == len(geo_dictionary.items()):
        for i, j in value[0].items():
          if i == 'geometry':
            bbox_coordinate = [[transform_x1, transform_y1], [transform_x2, transform_y2],
                                [transform_x3, transform_y3], [transform_x4, transform_y4]]
            j['coordinates'][0].append(bbox_coordinate)

  return geo_dictionary

In [ ]:
def make_geojson_dictionary(bbox_path, image_path):
  geo_dictionary = init_geo_dictionary()
  result = init_bbox_coordinate(bbox_path)
  geojson_dictionary = make_geo_dictionary(image_path, result, geo_dictionary)
  return geojson_dictionary

In [ ]:
def save_geojson_dictionary(bbox_paths, image_paths, json_paths):
  for bbox_path, image_path, json_path in zip(bbox_paths, image_paths, json_paths):
    geojson_dictionary = make_geojson_dictionary(bbox_path, image_path)
    with open(json_path, 'w') as f:
      geojson.dump(geojson_dictionary, f) 

In [ ]:
bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/_infer/_infer_jsk/fair1m_roi-trans_ms_rr_le90_train_half_without_airplane_0.3/', '*.log')
image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_train_half1_without_airplane_500/', bbox_paths)
geojson_paths = make_geojson_paths('/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3/', image_paths)

In [ ]:
save_geojson_dictionary(bbox_paths, image_paths, geojson_paths)

In [ ]:
geojson_path = '/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3/11593.geojson'

with open(geojson_path) as f:
    geojson_file = geojson.load(f)

geojson_file_name = geojson_path.split("/")[-1]   
print(colored(geojson_file_name, attrs=['bold']))
print()
pprint.pprint(geojson_file, width=160)

11593.geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[-93.221572, 36.633992], [-93.22158, 36.633989], [-93.221548, 36.633986], [-93.22154, 36.633994]],
                                             [[-93.221915, 36.633992], [-93.221851, 36.633989], [-93.221845, 36.633986], [-93.221912, 36.633995]],
                                             [[-93.22041, 36.633992], [-93.220413, 36.633989], [-93.220392, 36.633986], [-93.220389, 36.633994]],
                                             [[-93.220781, 36.633992], [-93.220837, 36.633989], [-93.220837, 36.633986], [-93.220781, 36.633994]],
                                             [[-93.220944, 36.633992], [-93.221, 36.633989], [-93.221, 36.633986], [-93.220944, 36.633994]],
                                             [[-93.220526, 36.633992], [-93.220523, 36.633989], [-93.220502, 36.633986], [-93.220504, 36.633994]],
                             

#Class Bounding Box Coordinate

In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate'

In [ ]:
cd ..

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
!pip install geojson

In [ ]:
import os
import shutil
import pprint
from glob import glob
from termcolor import colored

In [ ]:
import re
import math
import geojson
import rasterio
from osgeo import gdal
import numpy as np

In [ ]:
def make_bbox_paths(folder_name, file_extension):
  
  bbox_paths = glob(folder_name + file_extension)
  bbox_paths = sorted(bbox_paths, reverse=False)
  bbox_paths = sorted(bbox_paths, key=len)

  return bbox_paths

In [ ]:
def make_image_paths(folder_name, bbox_paths):

  image_paths = []
  for bbox_path in bbox_paths:
    img_path = bbox_path.replace('tif.log', 'tif')
    image_path = folder_name + img_path.split('/')[-1]
    image_paths.append(image_path)

  return image_paths

In [ ]:
def make_class_geojson_paths(folder_name, image_paths):

  ship_geojson_paths = []
  vehicle_geojson_paths = []
  airplane_geojson_paths = []

  for image_path in image_paths:
    ship_bbox_geo = image_path.split('/')[-1].replace('.tif', '_ship.geojson')
    ship_bbox_geojson = folder_name + ship_bbox_geo
    ship_geojson_paths.append(ship_bbox_geojson)

    vehicle_bbox_geo = image_path.split('/')[-1].replace('.tif', '_vehicle.geojson')
    vehicle_bbox_geojson = folder_name + vehicle_bbox_geo
    vehicle_geojson_paths.append(vehicle_bbox_geojson)

    airplane_bbox_geo = image_path.split('/')[-1].replace('.tif', '_airplane.geojson')
    airplane_bbox_geojson = folder_name + airplane_bbox_geo
    airplane_geojson_paths.append(airplane_bbox_geojson)
    
  return ship_geojson_paths, vehicle_geojson_paths, airplane_geojson_paths

In [ ]:
def init_geo_dictionary():

  geo_dictionary = {
  "type": "FeatureCollection",
  "name": "fair1m",
  "crs": { "type": "name", "properties": { "name": "" } },
  "features": [
  { "type": "Feature", "properties": { "id": 1 }, "geometry": { "type": "MultiPolygon",
                                                              "coordinates": [ [ ] ] } }
  ]
  }
  
  return geo_dictionary

In [ ]:
def init_class_bbox_coordinate(bbox_path):

  ship_list = []
  vehicle_list = []
  airplane_list = []

  with open(bbox_path , 'r') as f:
    while True:
      line = f.readline()
      if not line: break
      a = re.findall(r"[-+]?\d*\.\d+|\d+", line)
      b = re.findall(r"[A-Za-z_]+", line)[-1]
      if b == 'ship':
        ship_list.append([float(a[i]) for i in range(len(a)) if (i%3 == 2) or (i%3 == 0) and (i != 0)])
      elif b == 'vehicle':
        vehicle_list.append([float(a[i]) for i in range(len(a)) if (i%3 == 2) or (i%3 == 0) and (i != 0)])
      elif b == 'airplane':
        airplane_list.append([float(a[i]) for i in range(len(a)) if (i%3 == 2) or (i%3 == 0) and (i != 0)])
    f.close()

    return ship_list, vehicle_list, airplane_list

In [ ]:
def digit_length(n):
    return int(math.log10(n)) + 1 if n else 0

In [ ]:
def make_class_geo_dictionary(image_path, result, geo_dictionary):
  
  ds = gdal.Open(image_path)
  metadata = gdal.Info(ds, format='json')

  pixel_size = metadata['size']
  cornerCoordinates = metadata['cornerCoordinates']
  pixel_h = metadata['size'][0]
  pixel_w = metadata['size'][1]

  lowerLeft = cornerCoordinates['lowerLeft']
  lowerRight = cornerCoordinates['lowerRight']
  upperLeft = cornerCoordinates['upperLeft']
  upperRight = cornerCoordinates['upperRight']

  latitude = lowerLeft[0]
  latitude2 = lowerRight[0]
  longtitude = upperLeft[1]
  longtitude2 = lowerLeft[1]
    
  wgs_h = abs(latitude - latitude2)
  wgs_w = abs(longtitude - longtitude2)

  epsg = re.findall(r'\d+', metadata['coordinateSystem']['wkt'])

  if digit_length(abs(latitude)) >= 5:
    geo_dictionary['crs']['properties']['name'] = 'EPSG:' + epsg[-1]
  
  elif digit_length(abs(latitude)) < 5:
     geo_dictionary['crs']['properties']['name'] = "urn:ogc:def:crs:OGC:1.3:CRS84"

  for each_bbox in result:
    x1 = each_bbox[0]; y1 = each_bbox[1]
    x2 = each_bbox[2]; y2 = each_bbox[3]
    x3 = each_bbox[4]; y3 = each_bbox[5]
    x4 = each_bbox[6]; y4 = each_bbox[7]

    transform_x1 = latitude + ((x1 / pixel_w) * wgs_w)
    transform_y1 = longtitude - ((y1 / pixel_h) * wgs_h)
    transform_x2 = latitude + ((x2 / pixel_w) * wgs_w)
    transform_y2 = longtitude - ((y2 / pixel_h) * wgs_h)    
    transform_x3 = latitude + ((x3 / pixel_w) * wgs_w)
    transform_y3 = longtitude - ((y3 / pixel_h) * wgs_h)
    transform_x4 = latitude + ((x4 / pixel_w) * wgs_w)
    transform_y4 = longtitude - ((y4 / pixel_h) * wgs_h)

    point_count = 0
    for key, value in geo_dictionary.items():
      point_count += 1
      if point_count == len(geo_dictionary.items()):
        for i, j in value[0].items():
          if i == 'geometry':
            bbox_coordinate = [[transform_x1, transform_y1], [transform_x2, transform_y2],
                                [transform_x3, transform_y3], [transform_x4, transform_y4]]
            j['coordinates'][0].append(bbox_coordinate)

  return geo_dictionary

In [ ]:
def make_class_geojson_dictionary(bbox_path, image_path):

  ship_list, vehicle_list, airplane_list = init_class_bbox_coordinate(bbox_path)  

  geo_dictionary = init_geo_dictionary()
  ship_geojson_dictionary = make_class_geo_dictionary(image_path, ship_list, geo_dictionary)

  geo_dictionary = init_geo_dictionary()
  vehicle_geojson_dictionary = make_class_geo_dictionary(image_path, vehicle_list, geo_dictionary)

  geo_dictionary = init_geo_dictionary()
  airplane_geojson_dictionary = make_class_geo_dictionary(image_path, airplane_list, geo_dictionary)

  return ship_geojson_dictionary, vehicle_geojson_dictionary, airplane_geojson_dictionary

In [ ]:
def save_class_geojson_dictionary(bbox_paths, image_paths, ship_json_paths, vehicle_json_paths, airplane_json_paths):

  for bbox_path, image_path, ship_json_path, vehicle_json_path, airplane_json_path \
  in zip(bbox_paths, image_paths, ship_json_paths, vehicle_json_paths, airplane_json_paths):

    ship_geojson_dictionary, vehicle_geojson_dictionary, airplane_geojson_dictionary \
     = make_class_geojson_dictionary(bbox_path, image_path)

    with open(ship_json_path, 'w') as f:
      geojson.dump(ship_geojson_dictionary, f)   
    
    f.close()

    with open(vehicle_json_path, 'w') as f:
      geojson.dump(vehicle_geojson_dictionary, f)  

    f.close()

    with open(airplane_json_path, 'w') as f:
      geojson.dump(airplane_geojson_dictionary, f)    
    
    f.close()

In [ ]:
bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/_infer/_infer_jsk/fair1m_roi-trans_ms_rr_le90_train_half_without_airplane_0.3/', '*.log')
image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_train_half1_without_airplane_500/', bbox_paths)
ship_geojson_paths, vehicle_geojson_paths, airplane_geojson_paths = make_class_geojson_paths('/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3_class/', image_paths)

In [ ]:
save_class_geojson_dictionary(bbox_paths, image_paths, \
                              ship_geojson_paths, vehicle_geojson_paths, airplane_geojson_paths)

In [ ]:
ship_geojson_path = '/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3_class/11593_ship.geojson'
vehicle_geojson_path = '/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3_class/11593_vehicle.geojson'
airplane_geojson_path = '/content/drive/MyDrive/Aiffelthon/qgis/train_super_half1_without_airplane_0.3_class/11593_airplane.geojson'

with open(ship_geojson_path) as f:
    ship_geojson_file = geojson.load(f)

with open(vehicle_geojson_path) as f:
    vehicle_geojson_file = geojson.load(f)

with open(airplane_geojson_path) as f:
    airplane_geojson_file = geojson.load(f)

ship_geojson_file_name = ship_geojson_path.split("/")[-1]   
vehicle_geojson_file_name = vehicle_geojson_path.split("/")[-1]   
airplane_geojson_file_name = airplane_geojson_path.split("/")[-1]   

print(colored(ship_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(ship_geojson_file, width=160)

print("\n" * 2)

print(colored(vehicle_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(vehicle_geojson_file, width=160)

print("\n" * 2)

print(colored(airplane_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(airplane_geojson_file, width=160)

11593_ship.geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[-93.221572, 36.633992], [-93.22158, 36.633989], [-93.221548, 36.633986], [-93.22154, 36.633994]],
                                             [[-93.221915, 36.633992], [-93.221851, 36.633989], [-93.221845, 36.633986], [-93.221912, 36.633995]]]],
                            'type': 'MultiPolygon'},
               'properties': {'id': 1},
               'type': 'Feature'}],
 'name': 'fair1m',
 'type': 'FeatureCollection'}



11593_vehicle.geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[-93.22041, 36.633992], [-93.220413, 36.633989], [-93.220392, 36.633986], [-93.220389, 36.633994]],
                                             [[-93.220781, 36.633992], [-93.220837, 36.633989], [-93.220837, 36.633986], [-93.220781, 36.633994]],
                       